In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Dropout
import re


df = pd.read_csv('/content/drugLibTrain_raw_new.tsv', sep="\t", encoding='latin1')

df = df[['urlDrugName', 'benefitsReview', 'sideEffectsReview', 'commentsReview', 'condition']]

df = df.dropna()

df['all_reviews'] = df['benefitsReview'] + ' ' + df['sideEffectsReview'] + ' ' + df['commentsReview']

# 텍스트 전처리: 알파벳 이외의 문자 제거, 소문자 변환
df['all_reviews'] = df['all_reviews'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))
df['all_reviews'] = df['all_reviews'].str.lower()

df['condition'] = df['condition'].str.lower()
# 토큰화: 단어별로 분리
df['all_reviews'] = df['all_reviews'].apply(lambda x: x.split())

In [ ]:
#condition_counts = df['condition'].value_counts()

# 빈도수가 3개 이상인 항목만 선택하여 새로운 DataFrame 생성
#filtered_df = df[df['condition'].isin(condition_counts[condition_counts > 3].index)]

# 원본 DataFrame을 필터링된 DataFrame으로 업데이트
#df = filtered_df.copy()

In [ ]:
df['condition'].value_counts().sum()

3006

In [ ]:
condition_counts = df['condition'].value_counts()

# 개수가 2 이상인 값들의 인덱스 추출
condition_indices = condition_counts[condition_counts >= 2].index

# 조건을 만족하는 행들만 남기기
filtered_df = df[df['condition'].isin(condition_indices)]

In [ ]:
filtered_df.value_counts().sum()

TypeError: unhashable type: 'list'

In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 34.7 MB/s eta 0:00:00


In [ ]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 9.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt
import nltk

nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
import spacy

spacy.load('en_core_web_md')
stopwords1 = spacy.lang.en.stop_words.STOP_WORDS
stopwords1 = set(stopwords1)

stop_words_list = stopwords.words('english') + list(stopwords1)
stop_words = set(stop_words_list)
updated_stopwords = stopwords1

In [ ]:
def remove_stopwords(text):
    filtered_words = [word for word in text if word.lower() not in updated_stopwords]
    return filtered_words

df['all_reviews'] = df['all_reviews'].apply(remove_stopwords)

In [ ]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

df['all_reviews'] = df['all_reviews'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

In [ ]:
!pip install beautifulsoup4

In [ ]:
from nltk.stem.snowball import SnowballStemmer
from bs4 import BeautifulSoup

stemmer = SnowballStemmer('english')

def review_to_words(raw_review):
    if isinstance(raw_review, str):
        review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
        # 2. 공백 만들기
        letters_only = re.sub('[^a-zA-Z]', ' ', review_text)
        # 3. 소문자로 변경
        words = letters_only.lower().split()
        stemming_words = [stemmer.stem(w) for w in words]
        return ' '.join(stemming_words)
    else:
        # 문자열이 아닌 경우 원래의 값을 그대로 반환하여 비어 있지 않도록 함
        return raw_review


df['all_reviews']=df['all_reviews'].apply(review_to_words)

In [ ]:
df.head()

,urlDrugName,benefitsReview,sideEffectsReview,commentsReview,condition,all_reviews
0,enalapril,slowed the progression of left ventricular dys...,"cough, hypotension , proteinuria, impotence , ...","monitor blood pressure , weight and asses for ...",management of congestive heart failure,"[slowed, progression, left, ventricular, dysfu..."
1,ortho-tri-cyclen,Although this type of birth control has more c...,"Heavy Cycle, Cramps, Hot Flashes, Fatigue, Lon...","I Hate This Birth Control, I Would Not Suggest...",birth prevention,"[type, birth, control, cons, pros, help, cramp..."
2,ponstel,I was used to having cramps so badly that they...,Heavier bleeding and clotting than normal.,I took 2 pills at the onset of my menstrual cr...,menstrual cramps,"[having, cramps, badly, leave, balled, bed, da..."
3,prilosec,The acid reflux went away for a few months aft...,"Constipation, dry mouth and some mild dizzines...",I was given Prilosec prescription at a dose of...,acid reflux,"[acid, reflux, went, away, months, days, drug,..."
4,lyrica,I think that the Lyrica was starting to help w...,I felt extremely drugged and dopey. Could not...,See above,fibromyalgia,"[think, lyrica, starting, help, pain, sideeffe..."


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['all_reviews'])

review_sequences = tokenizer.texts_to_sequences(df['all_reviews'])

max_length = max([len(seq) for seq in review_sequences])
df_review_padded = pad_sequences(review_sequences, maxlen=max_length)

In [ ]:
df.head()

,urlDrugName,benefitsReview,sideEffectsReview,commentsReview,condition,all_reviews
0,165,slowed the progression of left ventricular dys...,"cough, hypotension , proteinuria, impotence , ...","monitor blood pressure , weight and asses for ...",778,"[949, 1758, 235, 6364, 1487, 3900, 116, 1759, ..."
1,316,Although this type of birth control has more c...,"Heavy Cycle, Cramps, Hot Flashes, Fatigue, Lon...","I Hate This Birth Control, I Would Not Suggest...",218,"[421, 286, 70, 2646, 2109, 38, 375, 54, 951, 4..."
2,341,I was used to having cramps so badly that they...,Heavier bleeding and clotting than normal.,I took 2 pills at the onset of my menstrual cr...,800,"[375, 1311, 764, 6376, 105, 1, 6377, 335, 7, 7..."
3,349,The acid reflux went away for a few months aft...,"Constipation, dry mouth and some mild dizzines...",I was given Prilosec prescription at a dose of...,11,"[163, 259, 44, 73, 12, 1, 5, 433, 19, 317, 46,..."
4,258,I think that the Lyrica was starting to help w...,I felt extremely drugged and dopey. Could not...,See above,480,"[91, 885, 150, 38, 7, 566, 30, 160, 23, 174, 1..."


In [ ]:
label_encoder_condition = LabelEncoder()

y_condition = label_encoder_condition.fit_transform(df['condition'])
# y_drugName = label_encoder_drug.fit_transform(df['drugName'])
y_condition = pd.get_dummies(y_condition).values

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(df_review_padded,y_condition,test_size=0.2,random_state=42)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

input_layer = Input(shape=(max_length,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128)(input_layer)
lstm_layer = LSTM(64)(embedding_layer)
condition_output = Dense(len(label_encoder_condition.classes_), activation='softmax', name='condition_output')(lstm_layer)
# drug_output = Dense(len(label_encoder_drug.classes_), activation='softmax', name='drug_output')(lstm_layer)
model = Model(inputs=input_layer, outputs=condition_output)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
38/38 [==============================] - 21s 432ms/step - loss: 7.0619 - accuracy: 0.0611 - val_loss: 6.6097 - val_accuracy: 0.1013
Epoch 2/10
38/38 [==============================] - 16s 414ms/step - loss: 6.3216 - accuracy: 0.0715 - val_loss: 6.8489 - val_accuracy: 0.1013
Epoch 3/10
38/38 [==============================] - 16s 414ms/step - loss: 6.1689 - accuracy: 0.0715 - val_loss: 6.9909 - val_accuracy: 0.1013
Epoch 4/10
38/38 [==============================] - 18s 467ms/step - loss: 6.1137 - accuracy: 0.0732 - val_loss: 7.1091 - val_accuracy: 0.1013
Epoch 5/10
38/38 [==============================] - 16s 434ms/step - loss: 6.0454 - accuracy: 0.0715 - val_loss: 7.1386 - val_accuracy: 0.1013
Epoch 6/10
38/38 [==============================] - 16s 432ms/step - loss: 5.9630 - accuracy: 0.0715 - val_loss: 7.2505 - val_accuracy: 0.1013
Epoch 7/10
38/38 [==============================] - 19s 491ms/step - loss: 5.8905 - accuracy: 0.0715 - val_loss: 7.3245 - val_accuracy: 0.1013